In [63]:
import pandas  as pd
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import *

from sklearn.model_selection import train_test_split
from json import loads

In [19]:
df = pd.read_csv("../datasets/Iris.csv")

In [20]:
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [21]:
df.columns

Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Species'],
      dtype='object')

In [85]:
feat = df[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']].values
labels = pd.get_dummies(df[['Species']]).values

In [86]:
X,x,Y,y = train_test_split(feat,labels,shuffle=True,)

In [87]:
inp = Input(shape=X.shape[1:],name='input_layer')
dense = Dense(8,)(inp)
out = Dense(Y.shape[-1],activation='softmax',name='output_layer')(dense)

model = Model(inp,out)

In [88]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [89]:
model.fit(
    x=X,
    y=Y,
    batch_size=4,
    epochs=100,
    validation_data=(x,y)
)

Epoch 1/100
28/28 [==============================] - 1s 8ms/step - loss: 1.0432 - accuracy: 0.2779 - val_loss: 0.9527 - val_accuracy: 0.1316
Epoch 2/100
28/28 [==============================] - 0s 2ms/step - loss: 0.9219 - accuracy: 0.3009 - val_loss: 0.8972 - val_accuracy: 0.4211
Epoch 3/100
28/28 [==============================] - 0s 2ms/step - loss: 0.8877 - accuracy: 0.5660 - val_loss: 0.8616 - val_accuracy: 0.5526
Epoch 4/100
28/28 [==============================] - 0s 2ms/step - loss: 0.8537 - accuracy: 0.5559 - val_loss: 0.8391 - val_accuracy: 0.5263
Epoch 5/100
28/28 [==============================] - 0s 2ms/step - loss: 0.8204 - accuracy: 0.4704 - val_loss: 0.8076 - val_accuracy: 0.5789
Epoch 6/100
28/28 [==============================] - 0s 2ms/step - loss: 0.7895 - accuracy: 0.6045 - val_loss: 0.7807 - val_accuracy: 0.6316
Epoch 7/100
28/28 [==============================] - 0s 2ms/step - loss: 0.7825 - accuracy: 0.6071 - val_loss: 0.7627 - val_accuracy: 0.6579
Epoch 8/100
2

Epoch 59/100
28/28 [==============================] - 0s 2ms/step - loss: 0.2451 - accuracy: 0.9178 - val_loss: 0.2426 - val_accuracy: 0.9474
Epoch 60/100
28/28 [==============================] - 0s 2ms/step - loss: 0.2677 - accuracy: 0.9701 - val_loss: 0.2399 - val_accuracy: 0.9474
Epoch 61/100
28/28 [==============================] - 0s 2ms/step - loss: 0.2630 - accuracy: 0.9802 - val_loss: 0.2371 - val_accuracy: 0.9474
Epoch 62/100
28/28 [==============================] - 0s 2ms/step - loss: 0.2217 - accuracy: 0.9754 - val_loss: 0.2332 - val_accuracy: 0.9737
Epoch 63/100
28/28 [==============================] - 0s 2ms/step - loss: 0.2186 - accuracy: 0.9710 - val_loss: 0.2285 - val_accuracy: 0.9474
Epoch 64/100
28/28 [==============================] - 0s 2ms/step - loss: 0.2603 - accuracy: 0.9588 - val_loss: 0.2244 - val_accuracy: 0.9474
Epoch 65/100
28/28 [==============================] - 0s 2ms/step - loss: 0.2237 - accuracy: 0.9715 - val_loss: 0.2232 - val_accuracy: 0.9737
Epoch 

In [90]:
model_config = loads(model.to_json())

In [260]:
examples = [
    {
        'input_layer':x.reshape(-1,4),
        'output_layer':y.argmax()
    }
    for
        x,y
    in
        zip(X[:10],Y[:10])
]

node_config = {
    'input_nodes':[
        {
            'name':'input_layer',
            'type':'row-named',
            'keys':['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
        }
    ],
    'output_nodes':[
        {
            'name':'output_layer',
            'type':'class-softmax',
            'classes':['Species_Iris-setosa', 'Species_Iris-versicolor','Species_Iris-virginica']
        }
    ],
}

In [265]:
config_names = ['units']

network = {}
layers = {}

for layer in model_config['config']['layers']:
    la = {}
    name = layer['config']['name']
    
    la['level'] = 0
    la['inbound_nodes'] = [l[0] for l in layer['inbound_nodes'][0]] if layer['inbound_nodes'] else []
    la['outbound_nodes'] = []
    la['config'] = { key:layer['config'][key] for key in config_names if key in layer['config']}
    
    layers[name] = model.get_layer(name)
    network[name] = la
    
for l in network:
    for n in network[l]['inbound_nodes']:
        network[n]['outbound_nodes'].append(l)

In [266]:
def set_level(input_nodes=[],level=0,network={},max_level = {0,}):
    for node in input_nodes:
        network[node]['level'] = level
        set_level(
            network[node]['outbound_nodes'],
            level+1,
            network,max_level
        )
        max_level.update({level,})
    return max(max_level)+1
    
input_nodes = [n['name'] for n in node_config['input_nodes']]
level = 0

max_level = set_level(input_nodes,level,network) 

In [267]:
levels = [[] for i in range(max_level)]

for layer in network:
    levels[network[layer]['level']].append(layer)

In [282]:
example = examples[0].copy()

In [322]:
outputs = {}

for level in levels:
    for layer in level:
        input_node = network[layer]['inbound_nodes']
        if len(input_node):
            if len(input_node) > 1:
                input_tensor = [outputs[t] for t in input_node]
            else:
                input_tensor = outputs[input_node[0]]
            outputs[layer] = layers[layer](input_tensor)
        else:
            input_tensor = example[layer]
            outputs[layer] = layers[layer](input_tensor)

In [321]:
outputs

{'input_layer': <tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[6. , 3.4, 4.5, 1.6]], dtype=float32)>,
 'dense_13': <tf.Tensor: shape=(1, 8), dtype=float32, numpy=
 array([[ 0.80223036, -5.1013756 ,  6.378769  ,  0.7620243 , -5.8473425 ,
         -0.63265145,  2.1279974 ,  2.745326  ]], dtype=float32)>,
 'output_layer': <tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.00502564, 0.6749734 , 0.32000095]], dtype=float32)>}

In [299]:
words = list('abcdefghijklmnopqrstuvwxyz')

enc = {j:i for i,j in enumerate(words)}
dec = {i:j for i,j in enumerate(words)}

In [317]:
text = 'hello'
key = 7

''.join([dec[w] for w in [(enc[w]+key)%26 for w in list(text)]])

'olssv'